In [4]:
import nltk
from nltk.stem import WordNetLemmatizer
import argparse
import pandas as pd
from sklearn.svm import LinearSVC
from sklearn.feature_extraction import DictVectorizer
import pandas as pd
from sklearn.metrics import classification_report
import argparse
import numpy as np

In [20]:
from sklearn.datasets import load_digits
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
import sklearn.metrics as mx

Function for the extraction of differenct features

In [5]:
#inspire from from different documentations and stack over flow questions

def wordnet_pos(pos):

    if pos.startswith('J'):
        return 'a'
    elif pos.startswith('V'):
        return 'v'
    elif pos.startswith('N'):
        return 'n'
    elif pos.startswith('R'):
        return 'r'
    else:
        return 'n'

def lemma_extraction(tokens, pos_list):
    lemmas = []
    lem = WordNetLemmatizer()
    for token, pos in zip(tokens, pos_list):
        lemma = lem.lemmatize(token, wordnet_pos(pos))
        lemmas.append(lemma)


    return lemmas

def pos_extraction(tokens):
    pos_list = []
    tagged = nltk.pos_tag(tokens)
    for token, pos_tag in tagged:
        pos_list.append(pos_tag)   

    return pos_list

def previous_and_next_token_extraction(tokens):
    position_index = 0

    prev_tokens = []
    next_tokens = []

    for i in range(len(tokens)):

        prev_index = (position_index - 1)
        next_index = (position_index + 1)

        #previous token
        if prev_index < 0:
            previous_token = "None"
        else: 
            previous_token = tokens[prev_index]

        prev_tokens.append(previous_token)

        #next token
        if next_index < len(tokens):
            next_token = tokens[next_index]
        else: 
            next_token = "None"

        next_tokens.append(next_token)

        #moving to next token in list 
        position_index += 1

    return prev_tokens, next_tokens



def neg(tokens, neg):
    neg_word_list = []

    for token in tokens:

        # label 1 if token is in negative word list
        if token in neg:
            label = 1

        # label 0 if token is not in negative word list
        else:
            label = 0
        
        neg_word_list.append(label)

    return neg_word_list

Extracting features with the help of above functions

In [6]:

def file(input_file):
    # open and red file and returns token, gold labels and chapters 
    
    df = pd.read_csv(input_file, encoding='utf-8', delimiter = "\t", quotechar = '|')
    chapters = df.iloc[:, 0]
    sent_id = df.iloc[:, 1]
    token_position = df.iloc[:, 2]
    tokens = df.iloc[:, 3]
    gold = df.iloc[:, 4]
    
    return tokens, gold, chapters, 

def write_features(tokens,gold):
    
    negl= ['nor', 'neither', 'without', 'nobody', 'none', 'nothing', 'never', 'not', 'no', 'nowhere', 'non']
    #not considering at this moment, will work on the smooth implimentation of this
    # written the code for negetion words, fucntion can be seen above
    # had few difificulties in final processing of output 
    
    
    # Defining header names
    feature_names = ["token",
                "lemma",
                "pos_tag",
                "prev_token",
                "next_token",
                    "neg"]

   # using all above mentioned functions for the extraction of features, except fucntion neg  
    pos_tag = pos_extraction(tokens)
    lemma = lemma_extraction(tokens, pos_tag)
    neg_word = neg(tokens,negl)
    prev_next_token = previous_and_next_token_extraction(tokens)
    prev_tokens, next_token = prev_next_token

    features_dict = {'token': tokens, 'pos_tag': pos_tag,'lemma': lemma,'neg_word':neg_word ,'prev_token': prev_tokens,
                     'next_token': next_token}
    return features_dict
        

Creatinf featues list, svm model and prdictions 

In [7]:

def data(input_file):
    # main function to add features to dataset
    data = []
    tokens, gold, chapters= file(input_file) # from def file mentioned above
    feature_dict = write_features(tokens,gold) # from above function
    #features 
    token = feature_dict['token']
    lemma = feature_dict['lemma']
    pos_tag = feature_dict['pos_tag']
    neg_word =  feature_dict['neg_word'],
    prev_token= feature_dict['prev_token']
    next_token= feature_dict['next_token']
    golds = gold.tolist() #gold labels to list
    token = token.tolist() #tokens to list 
    
    #print(type(next_token)
    #,type(token),type(lemma),type(pos_tag),type(neg_word),
    # type(prev_token),type(next_token))
    #print(type(list(neg_word)))
    for token,lemma,pos_tag,prev_token,next_token in zip(token,lemma,pos_tag,
                                                                         prev_token,next_token):
        feature_dict = {"Tokens": token, 'lemmas':lemma,'pos_tags':pos_tag,
                        'prev_token':prev_token,'next_token':next_token}

        #feature_dict = {"1": token, '2':lemma,'3':pos_tag,'4':neg_word,'5':prev_token,'6':next_token, '7': gold}
        data.append(feature_dict)
        #print(data)
    return data, golds

def create_classifier(train_features, train_targets: list):
    #creating SVM model 
    
    model = LinearSVC()
    vec = DictVectorizer()
    featuresv = vec.fit_transform(training_features)
    train_targets = np.array(train_targets)
    model.fit(featuresv, train_targets)

    return model, vec



def predicted(testfile, model,vec):
    #prediction of SVM on test and dev set
    features, goldlabels = data(testfile)
    test_features_vectorized = vec.transform(features)
    predictions = model.predict(test_features_vectorized)

    return predictions, goldlabels


Importing Datasets

In [8]:
training_file = 'data/no_dev.txt'
dev_file = 'data/dev.txt'

In [9]:
#merging two test set, circle and cardboard into one 
test_card = pd.read_csv('data/test_1.txt', sep="\t", header = None)#cardboard test
test_circ = pd.read_csv('data/test_2.txt', sep="\t", header = None)#circle test

frames = [test_card, test_circ]
test_data = pd.concat(frames)
#test_data.rename(columns={1: 'phrase_id', 0: 'chapter_id', 2:'word_id', 3:'word', 4:'label'}, inplace=True)
test_data.to_csv('merge_test_.txt', sep = '\t', index=False)

In [10]:
test_merge = "merge_test_.txt" # final test set

In [11]:
#generating TRAINING FEATURES 
training_features, gold_labels= data(training_file)


In [12]:
 #creating model and vectorzor 
model, vec= create_classifier(training_features, gold_labels)

In [13]:
# develoment predictions 
dev, gold = predicted(dev_file, model, vec)
# test predictions 
test1, gold1 = predicted(test_merge, model, vec)

In [14]:
#adding predictions to the df created by mearging the files. 
test_data['SVM__predictions'] = test1
test_data.rename(columns={1: 'sent_id', 0: 'chapter', 2:'word_id', 3:'token', 4:'gold'}, inplace=True)

Classification report and P,R,f-1scores

In [15]:
def print_confusion_matrix(predictions, goldlabels):
    # based on example from https://datatofish.com/confusion-matrix-python/
    data = {'Gold': goldlabels, 'Predicted': predictions}
    df = pd.DataFrame(data, columns=['Gold', 'Predicted'])

    confusion_matrix = pd.crosstab(df['Gold'], df['Predicted'], rownames=['Gold'], colnames=['Predicted'])
    print(confusion_matrix)
    report = classification_report(goldlabels,predictions,digits = 3)
    print('METRICS: ')
    print()
    print(report)

Development se classification report

In [16]:
cf_dev = print_confusion_matrix(dev,gold)

Predicted  B-NEG  I-NEG      O
Gold                          
B-NEG        149      0     27
I-NEG          0      2      1
O              6      0  13381
METRICS: 

              precision    recall  f1-score   support

       B-NEG      0.961     0.847     0.900       176
       I-NEG      1.000     0.667     0.800         3
           O      0.998     1.000     0.999     13387

    accuracy                          0.997     13566
   macro avg      0.986     0.838     0.900     13566
weighted avg      0.997     0.997     0.997     13566



Test set classification report

In [17]:
cf_test = print_confusion_matrix(test1,gold1)

Predicted  B-NEG      O
Gold                   
B-NEG        247     22
I-NEG          0      5
O             13  18929
METRICS: 

              precision    recall  f1-score   support

       B-NEG      0.950     0.918     0.934       269
       I-NEG      0.000     0.000     0.000         5
           O      0.999     0.999     0.999     18942

    accuracy                          0.998     19216
   macro avg      0.650     0.639     0.644     19216
weighted avg      0.998     0.998     0.998     19216



/Users/rishvikchandel/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/rishvikchandel/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/rishvikchandel/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(r

Error analysis on test set 

In [18]:
def error_detect(tokens, gold, chapters, sent_id, pred):
    #write all the errors in a tsv file 
    #take inouts from test set and give error examples 
    error_dict = []
    for token, gol, chap,sent, pre in zip(tokens, gold,chapters, sent_id, pred):
        if gol != pre:
            error_sent = {"chap": chap, "sent_id": sent, "Token": token, "Gold": gol, "Prediction": pre}
            error_dict.append(error_sent)
            df= pd.DataFrame(error_dict)
            df.to_csv('error_test.tsv', sep='\t', header = True)  

In [19]:
error_detect(test_data['token'],test_data['gold'],test_data['chapter'],test_data['sent_id'],test_data['SVM__predictions'],)


Feature ablation

In [132]:
# Feature ablation using svm's svc 

def score_model(features_train, gold1):
    clf = SVC(gamma='scale', kernel='rbf') #dvc model
    vec = DictVectorizer()
    featuresv = vec.fit_transform(features_train)
    #featurest = vec.fit_transform(features_test)
    train_targets = np.array(gold1)
    clf = clf.fit(featuresv, train_targets)
   # pred = clf.predict(featurest)
   # return mx.accuracy_score(gold2, pred)
    return clf, vec

In [133]:
model1, vec1= score_model(training_features, gold_labels)

In [232]:
def prediction(merge, model,vec): 
    #prediction on full features 
    test, gold = data(merge)
    test_ = vec.transform(test)
    pred = model.predict(test_)
    #print(test_)
    return  mx.accuracy_score(gold, pred)


In [233]:
acc = prediction(test_merge, model1, vec1) # acc on all features 


In [284]:
#removing one feature at a time to check accuracy
print(test1[1])
columns= ['Tokens','lemmas','pos_tags',	'prev_token',	'next_token'] # all features 
for feature in columns:
    feature_importances = []
    test, gold = data(test_merge)
    #print(feature)
    
    for items in test:
       # print(items)
        if feature in  items:
            del items[feature]
            #X_test_ablated = X_test.drop(feature, axis=1)
    #print(test[1])
    test_ = vec.transform(test)
    y_pred_ablated = model.predict(test_)
    feature_importance1 = mx.accuracy_score(gold, y_pred_ablated) #accuracy after droping that feature 
    #print(feature_importance)
    # Calculate the difference in f1 score compared to the baseline
    feature_importance = mx1 - feature_importance1
    feature_importances.append(feature_importance)
    print(feature,feature_importance1,  feature_importances)
#print(test)

{}
Tokens 0.9907889258950874 [0.00603663613655292]
lemmas 0.9907889258950874 [0.00603663613655292]
pos_tags 0.9976582014987511 [-0.0008326394671107629]
prev_token 0.9977622814321399 [-0.0009367194004995527]
next_token 0.9971378018318068 [-0.00031223980016648056]


above mentioned is the feature removed, accuracy after removing that feature and acctual accuracy - accuracy after removing thath feature